Importamos la paqueteria necesaria

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

importamos desde nuestra cuenta de github datos del algoritmo de transporte

In [ ]:
#url = 'https://raw.githubusercontent.com/daerksun/MDyE---Algoritmo-de-Transporte/main/TransporteBalanceado.csv'
url = 'https://raw.githubusercontent.com/daerksun/MDyE---Algoritmo-de-Transporte/main/TransporteDesbalanceadoDemanda.csv'
#url = 'https://raw.githubusercontent.com/daerksun/MDyE---Algoritmo-de-Transporte/main/TransporteDesbalanceadoOferta.csv'
df = pd.read_csv(url)
print("Datos cargados:\n\n",df)

Datos cargados:

   Unnamed: 0  R1  R2  R3  R4  Oferta
0         S1  10   2  20  11      45
1         S2  12   7   9  20      55
2         S3   4  14  16  18      30
3    Demanda  25  35  35  20       0


Convertimos los datos a array para poder manipularlos con la libreria NumPy

In [ ]:
loaded_matrix = df.to_numpy()
corr = loaded_matrix[:,1:]
print("\n\nDatos en numpy array:\n\n",corr)



Datos en numpy array:

 [[10 2 20 11 45]
 [12 7 9 20 55]
 [4 14 16 18 30]
 [25 35 35 20 0]]


Verificamos si el sistema esta balanceado

In [ ]:
if (np.sum(corr[:-1,-1])-np.sum(corr[-1,:-1]) == 0):
  print("Balanceado")
elif (np.sum(corr[:-1,-1])-np.sum(corr[-1,:-1]) < 0):
  print("Balancear Oferta")
else:
  print("Balancear Demanda")


Balancear Demanda


Se obtiene el vector de coeficientes, por lo que se procede a concatenar los pesos que vamos a minimizar pasandolos de una matriz a un vector

In [ ]:
obj_coeff = np.zeros((0,0))
for i,x_val in enumerate(corr[:-1,0]):
  obj_coeff = np.concatenate((obj_coeff,corr[i,:-1]), axis = None)
print("Coeficientes de función objetivo:\n",obj_coeff)

Coeficientes de función objetivo:
 [10 2 20 11 12 7 9 20 4 14 16 18]


Posteriormente armamos la matriz de restricciónes, la cual contemplara las restricciones horizontales y verticales llevadas al mismo plano

In [ ]:
num_S = corr[:-1,0].size
num_R = corr[0,:-1].size

const_coeff = np.zeros((num_R + num_S, num_R * num_S))

for i,col in enumerate(corr[:-1,0]):
  for j,x_val in enumerate(corr[0,:-1]):
    const_coeff[i,j+(i*num_R)] = 1

for i,col in enumerate(corr[0,:-1]):
  for j,x_val in enumerate(corr[:-1,0]):
    const_coeff[i+num_S,(j*num_R)+i] = 1

print("\nCoeficientes de restricciones:\n",const_coeff)


Coeficientes de restricciones:
 [[1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.]
 [1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1.]]


In [ ]:
const_lims = np.concatenate((corr[:-1,-1],corr[-1,:-1]), axis = 0)

print("\nLímites de restricciones:\n",const_lims)


Límites de restricciones:
 [45 55 30 25 35 35 20]


In [ ]:
res2 = linprog(obj_coeff,A_eq=const_coeff,b_eq=const_lims,method='simplex',)
print("Función objetivo = ",res2.fun, "\n")

for j,x_val in enumerate(corr[0,:-1]):
  wrtcol = wrtcol + "\t|R"+str(j+1)
print(wrtcol)

for i,col in enumerate(corr[:-1,0]):
  wrtcol = "S"+str(i+1)
  for j,x_val in enumerate(corr[0,:-1]):
    wrtcol = wrtcol + "\t|" + str(res2.x[(i*num_R)+j])
  print("\t--->")
  print(wrtcol)
  wrtcol = ""
print("\t--->")

Función objetivo =  0.0 

	|R1	|R2	|R3	|R4
	--->
S1	|0.0	|0.0	|0.0	|0.0
	--->
S2	|0.0	|0.0	|0.0	|0.0
	--->
S3	|0.0	|0.0	|0.0	|0.0
	--->


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  """Entry point for launching an IPython kernel.
